In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.simplefilter("ignore")

In [2]:
prishvin = pd.read_csv("http://r.senyukhin.ru/files/prishvin.csv", sep='\t')
prishvin.head()

,id,text,date,dateTop,notDated,julian_calendar
0,17938,"<p>Вчера был наш русский Новый год, он старыми...",1914-01-14,0000-00-00,0,1
1,17939,<p>Зимний рассвет: строгий красный начался и н...,1914-01-19,0000-00-00,0,1
2,17940,"<p>Зимой рано утром, когда начинает светать, с...",1914-01-25,0000-00-00,0,1
3,17941,<p>Наблюдения в Песочках. Почтальон Николай бы...,1914-01-26,0000-00-00,0,1
4,17942,<p>Собрание Религиозно-философского общества д...,1914-02-01,0000-00-00,0,1


### Этап 1. Очистка данных

In [3]:
# Удаление тегов и следов маркдауна
patterns = [r'<com id=\d+"/>', r'<com id="\d+"/>', r'<\w+>', r'<\/\w+>', r'<\w+\s/>', '\*', '#']

for pattern in patterns:
    prishvin['text'] = prishvin['text'].str.replace(pattern, '', regex=True)

# Удаление следов плохой типографики
prishvin['text'] = prishvin['text'].str.replace(r'(\w+-)\s([го|я|й])', r'\1\2', regex=True)

# Я решил удалить следы раскрытия сокращений слов, например, М[ария]
prishvin['text'] = prishvin['text'].str.replace(r'(\w)\[(\w+)\]', r'\1\2', regex=True)


[Ручная очистка после получения готовых результатов]

In [4]:
prishvin['text'] = prishvin['text'].str.replace(r'([Рр])уко-пис', r'\1укопис', regex=True)

### Этап 2. Предобработка данных
Поскольку мы хотим изучить тенденцию частотности слов по годам, то я создам колонку с годом записи

In [6]:
prishvin['year'] = pd.to_datetime(prishvin['date'], format='%Y-%m-%d').dt.year
years = prishvin['year'].unique()

In [7]:
# Для тренировки построю модель для всего дневника
vectorizer = TfidfVectorizer(use_idf=True)
X = vectorizer.fit_transform(prishvin['text'])
prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
prishvin_words.head(10)

,TF-IDF
разбойники,0.173145
покаялись,0.167480
бебеля,0.141153
не,0.124583
свет,0.115568
то,0.115415
злодеяний,0.115296
хулиганов,0.106520
как,0.106390
хулиганство,0.104568


### Этап 3. Исследование по годам

In [8]:
years.sort()

prishvin_by_years = pd.DataFrame()

In [9]:
# Первое приближение 
for year in years:
    vectorizer = TfidfVectorizer(use_idf=True)
    X = vectorizer.fit_transform(prishvin[prishvin['year'] == year]['text'].to_list())
    prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
    prishvin_by_years[year] = prishvin_words.head(10).index

In [10]:
prishvin_by_years

,1905,1906,1908,1909,1910,1911,1912,1913,1914,1915,...,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954
0,не,не,сознания,письмо,он,зотов,не,не,не,сделала,...,роли,валяюсь,поэт,кто,брился,траур,правда,друг,стойло,лучшими
1,но,нет,жизни,что,на,трудовик,конец,что,свет,завещание,...,личности,визитом,пришел,на,вчерашняя,письмо,воплощение,барвихи,баранов,годом
2,бы,теперь,должно,не,не,фокусы,что,жизни,то,его,...,истории,готовит,сторожку,распад,мылся,что,подтвердила,проболтали,важнее,сидел
3,ее,на,жизнь,вдруг,так,жена,но,могу,как,видимому,...,заявка,метелицей,ствовал,портрет,шаховы,не,шилькреды,среда,год,пожеланиями
4,что,все,для,написал,что,душе,когда,помню,разбойники,случайные,...,сережа,эстонии,женои,борется,болтали,романа,бразная,годом,стойлом,весной
5,мне,он,что,было,все,так,на,мария,на,съездила,...,требование,племянница,яковлевич,попробовал,серая,меньше,образах,хоронить,стойлу,пантелеймону
6,она,это,вечной,письма,то,от,любовь,ивановна,что,хлопотать,...,что,встречал,лепесток,не,горячая,ему,мечта,будками,бараны,писателя
7,на,то,недостроенная,меня,его,нельзя,мысли,дв,год,наследников,...,нравственного,чашу,луговскои,простой,спеша,год,звезд,отправляется,время,принимал
8,от,бог,назвал,то,говорит,черносотенный,так,послать,бебеля,неловко,...,себе,мирскую,симпатичная,человек,ни,так,поели,тянуть,году,месте
9,кажется,никогда,ругали,ее,за,хе,все,но,покаялись,начала,...,право,возобновился,удинцевым,свете,весело,моего,мечтал,выздоровления,когда,кондратьевичу


Результаты содержат много [стоп-слов](https://ru.wikipedia.org/wiki/Шумовые_слова). Предлагаю от них избавиться.

Я взял список стоп-слов отсюда: https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt?ysclid=ldoas1im8w726822300

In [11]:
import requests

data = requests.get('http://r.senyukhin.ru/files/stopwords-ru.txt')
data.encoding = 'utf-8'
stop_words = data.text.split()


for year in years:
    vectorizer = TfidfVectorizer(use_idf=True, stop_words=stop_words)
    X = vectorizer.fit_transform(prishvin[prishvin['year'] == year]['text'].to_list())
    prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
    prishvin_by_years[year] = prishvin_words.head(10).index


In [12]:
prishvin_by_years.loc[:, '1920':'1930']

,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930
0,раздирает,парижский,ценз,маркса,рукопись,судьба,трактир,алпатов,снизу,засыпаны,спустят
1,товарами,средняя,личность,лев,рукописью,судьбу,провинции,франс,снегом,пороши,колокол
2,ситцы,достоевск,спеца,вагоне,руку,жизни,деле,генерала,окна,летят,известно
3,привычками,подросток,кумир,социализм,беда,творчества,стороны,оболочка,узорах,наверное,барельефных
4,наследственными,романа,вождь,дамочка,годы,вызываешь,расплатится,социализм,намекая,леса,спустить
5,приказчик,любили,масса,львов,света,вызывает,связанным,человечеству,стеклах,10,носы
6,рядов,чтения,социализма,талдом,тверском,ба,беднотой,учение,подоконников,провозглашает,ервого
7,надрезать,любила,цензу,москве,сунула,некотором,образах,любовь,пекину,провода,тяпнуты
8,лоно,период,строя,интеллигента,рельсам,пример,стакан,яйца,подбило,проводили,еликой
9,ситцев,свете,понятие,мужчиной,зубакин,ведет,общегражданском,жизни,морозных,проводите,навозный


Вообще примечательно, что год от года слова практически не повторяются, а часть этих лемм — это разные формы. 

# Модификация эксперимента
Попробуем прийти к начальным формам слов

In [13]:
import pymorphy2
from razdel import tokenize

def normalize_tokens(entry):
    tokens = [_.text for _ in tokenize(entry)] 
    morph = pymorphy2.MorphAnalyzer()
    res = []

    for token in tokens:
        res.append(morph.parse(token)[0].normal_form)
    
    return ' '.join(res)


In [14]:
def normalize_tokens_1(entry):
    from natasha import Doc, MorphVocab, NewsMorphTagger, NewsEmbedding, Segmenter

    doc = Doc(entry)
    segmenter = Segmenter()
    doc.segment(segmenter)
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    morph_vocab = MorphVocab()
    doc.tag_morph(morph_tagger)

    for token in doc.tokens: token.lemmatize(morph_vocab)

    return ' '.join([_.lemma for _ in doc.tokens])

In [150]:
#p = prishvin.sample(10)
prishvin['tokens'] = prishvin['text'].apply(normalize_tokens_1)
prishvin.to_csv('tokenizer.csv', sep='\t')

In [15]:
prishvin_tokenized = pd.read_csv("tokenizer.csv", sep='\t')
prishvin_tokenized

,index_old id,text,date,dateTop,notDated,julian_calendar,year,tokens
0,17938,"Вчера был наш русский Новый год, он старыми с...",1914-01-14,0000-00-00,0,1,1914,"вчера быть наш русский новый год , он старый с..."
1,17939,Зимний рассвет: строгий красный начался и не ...,1914-01-19,0000-00-00,0,1,1914,зимний рассвет : строгий красный начаться и не...
2,17940,"Зимой рано утром, когда начинает светать, сне...",1914-01-25,0000-00-00,0,1,1914,"зима рано утро , когда начинать светать , снег..."
3,17941,Наблюдения в Песочках. Почтальон Николай был ...,1914-01-26,0000-00-00,0,1,1914,наблюдение в песочек . почтальон николай быть ...
4,17942,Собрание Религиозно-философского общества для...,1914-02-01,0000-00-00,0,1,1914,собрание религиозно-философский общество для и...
...,...,...,...,...,...,...,...,...
9942,539117,"Пишу о доме на колесах, пишу, убиваюсь, умере...",1939-12-28,0000-00-00,0,0,1939,"писать о дом на колесо , писать , убиваться , ..."
9943,539118,"Вчера были Удинцев, Попов и Клавдия Борисовна...",1939-12-29,0000-00-00,0,0,1939,"вчера быть удинец , попов и клавдия борисович ..."
9944,539119,"В магазинах нет ничего, даже сахару, десятки ...",1939-12-30,0000-00-00,0,0,1939,"в магазин нет ничто , даже сахара , десяток ты..."
9945,539120,Все мое писательство в совокупности есть любо...,1939-12-31,0000-00-00,0,0,1939,весь мой писательство в совокупность быть любо...


In [16]:
# Повторим TF-IDF анализ, но теперь результат будет в формате wide2long
import requests

data = requests.get('http://r.senyukhin.ru/files/stopwords-ru.txt')
data.encoding = 'utf-8'
stop_words = data.text.split()

years = prishvin_tokenized['year'].unique()
years.sort()
prishvin_by_years = pd.DataFrame(columns=['TF-IDF', 'word', 'year'])


for year in years:
    vectorizer = TfidfVectorizer(use_idf=True, stop_words=stop_words)
    X = vectorizer.fit_transform(prishvin_tokenized[prishvin_tokenized['year'] == year]['tokens'].to_list())
    prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
    prishvin_words['word'] = prishvin_words.index
    prishvin_words['year'] = year
    prishvin_words.reset_index(inplace=False)
    prishvin_by_years = prishvin_by_years.append(prishvin_words.head(20), ignore_index=True)

prishvin_by_years

,TF-IDF,word,year
0,0.226134,похожий,1905
1,0.226134,чувство,1905
2,0.226134,сойтись,1905
3,0.226134,любовь,1905
4,0.150756,яша,1905
...,...,...,...
975,0.000000,праздник,1954
976,0.000000,представить,1954
977,0.000000,премия,1954
978,0.000000,препровождение,1954


In [61]:
len(years)

49

In [68]:
import seaborn as sns
import matplotlib.pyplot as plt

#dataset = sns.load_dataset(prishvin_by_years)

#grid = sns.FacetGrid(prishvin_by_years, col='year', col_wrap=4)

#grid.map(sns.barplot, "TF-IDF", "word")

fig, axes = plt.subplots(7, 7, sharex=True, figsize=(100,100))
counter = 0
for year in years:
    sns.barplot(ax=axes.flatten()[counter], x='TF-IDF', y='word', data=prishvin_by_years[prishvin_by_years['year'] == year])
    axes.flatten()[counter].set_title(year)
    counter += 1
    